In [1]:
from typing import Iterator

from openpilot.tools.lib.logreader import LogReader

IDENTIFIERS = [
  "0b9d9ff7c65286db/00000004--52d36c6566",
  "0b9d9ff7c65286db/00000005--db16a09ea1",
]


def collect_can_frames(identifier: str, address: int, bus: int) -> Iterator[tuple[int, bytes]]:
  t0 = None
  for evt in LogReader(identifier):
    which = evt.which()
    if which != 'can':
      if t0 is None and which != 'initData':
        t0 = evt.logMonoTime
      continue
    for can in evt.can:
      if can.address != address:
        continue
      if can.src != bus:
        continue
      t = (evt.logMonoTime - t0) / 1e9
      yield t, can.dat


LANE_KEEP_ASSIST = 0x3F2
BUS = 2

frames = []
for identifier in IDENTIFIERS:
  frames.extend(collect_can_frames(identifier, LANE_KEEP_ASSIST, BUS))

len(frames)

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


26370

In [2]:
# dat[1] & 0xF0 is checksum (4 bits)
def strip_checksum(dat: bytes) -> tuple[bytearray, int]:
  dat = bytearray(dat)
  checksum = (dat[1] & 0xF0) >> 4
  dat[1] &= 0x0F
  return dat, checksum

frames_with_checksum = [(t, *strip_checksum(dat)) for t, dat in frames]

In [3]:
import pandas as pd

df = pd.DataFrame(frames_with_checksum, columns=['t', 'dat', 'checksum']).drop_duplicates(subset=['dat'])
df.head()

,t,dat,checksum
0,-0.443376,"[0, 0, 0, 0, 8, 0, 1, 110]",14
1,-0.394022,"[0, 1, 0, 0, 8, 0, 1, 106]",1
2,-0.344220,"[0, 2, 0, 0, 8, 0, 1, 106]",0
3,-0.293509,"[0, 3, 0, 0, 8, 0, 1, 106]",15
4,-0.244258,"[0, 4, 0, 0, 8, 0, 1, 106]",14


In [4]:
def get_checksum(dat: bytes, addr: int) -> int:
  byte_index = {
    0x2ED: 1,
    0x2F5: 1,
    0x305: 4,
    0x38D: 5,
    0x3CD: 7,
    0x3F2: 1,
    0x5CD: 1,
  }
  low_nibble = addr in (0x38D, )

  if addr not in byte_index:
    raise ValueError(f"Unhandled address {addr}")

  byte = dat[byte_index[addr]]
  return byte & 0xF if low_nibble else byte >> 4

# assume checksum is zeroed out
def calculate_checksum(dat: bytes, addr: int) -> int:
  init = {
    0x3F2: 0xD,
  }
  if addr not in init:
    raise ValueError(f"Unhandled address {addr}")
  agg = init[addr]
  for i in range(8):
    agg += (dat[i] & 0xF) + (dat[i] >> 4)
  return (8 - agg) & 0xF

In [5]:
calculated_checksum = df['dat'].apply(lambda dat: calculate_checksum(dat, LANE_KEEP_ASSIST))
correct = calculated_checksum == df['checksum']
correct_pct = correct.sum() / len(correct)
print(f"{correct_pct:.2%}")

100.00%
